In [5]:
import numpy as np
import pandas as ps
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense,Flatten,BatchNormalization,Conv2D,MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
%matplotlib inline

In [6]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print('Num GPUs Available: ', len(physical_devices))
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    print('No GPUs Available.')

No GPUs Available.


Preparing data

In [36]:
root_images_folder='D:\plant image classification\images'
root_directories=os.listdir(root_images_folder)
required_directories=[]

for i in root_directories :
    b=os.listdir(f'{root_images_folder}\{i}')
    if len(b)>=50:
        required_directories.append(f'{root_images_folder}\{i}')


'D:\\plant image classification\\images\\10040445'